In [79]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error # 回帰問題における性能評価に関する関数
from sklearn.metrics import log_loss, accuracy_score, precision_recall_fscore_support, confusion_matrix # 回帰問題における性能評価に関する関数
import seaborn as sns

In [83]:
#データ読み込み
df_ks = pd.read_csv("./ks-projects-201801.csv")

In [84]:
#データ概要
display( df_ks.head() )
display( df_ks.describe() )
display( df_ks.info() )

#各項目の値の種類
for key in df_ks:
    if 'ID' == key or 'name' == key:
        continue
    display( '<' + key + '>')
    display( df_ks[key].value_counts() )

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


,ID,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,1.074731e+09,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,6.190862e+08,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,5.382635e+08,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,1.075276e+09,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.610149e+09,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
ID                  378661 non-null int64
name                378657 non-null object
category            378661 non-null object
main_category       378661 non-null object
currency            378661 non-null object
deadline            378661 non-null object
goal                378661 non-null float64
launched            378661 non-null object
pledged             378661 non-null float64
state               378661 non-null object
backers             378661 non-null int64
country             378661 non-null object
usd pledged         374864 non-null float64
usd_pledged_real    378661 non-null float64
usd_goal_real       378661 non-null float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


None

'<category>'

...

0.00        68112
1.00         5341
25.00        3877
10.00        3624
50.00        3141
100.00       2674
5.00         2598
20.00        2473
30.00        1717
2.00         1451
15.00        1310
40.00        1189
60.00        1184
35.00        1179
75.00        1120
150.00       1065
200.00        989
125.00        917
70.00         791
55.00         777
110.00        773
45.00         758
11.00         736
80.00         711
26.00         706
120.00        666
6.00          665
250.00        652
500.00        619
300.00        604
            ...  
137.65          1
10355.12        1
53012.00        1
403.76          1
572.96          1
1394.74         1
154.45          1
155.55          1
316.40          1
2354.56         1
29406.05        1
87.14           1
53001.00        1
617.55          1
13249.50        1
3402.93         1
7350.70         1
10313.13        1
161.83          1
161.17          1
34095.50        1
26503.00        1
23.13           1
36238.11        1
13251.00  

'<usd_pledged_real>'

0.00         52527
1.00          6678
10.00         3633
25.00         3455
50.00         2937
5.00          2584
100.00        2461
20.00         2354
2.00          1700
30.00         1655
15.00         1280
60.00         1128
35.00         1126
40.00         1106
75.00         1032
150.00         961
200.00         906
11.00          905
125.00         838
26.00          832
6.00           774
55.00          743
70.00          740
110.00         720
51.00          688
45.00          683
80.00          657
120.00         624
3.00           623
500.00         611
             ...  
75392.17         1
13801.00         1
1178.03          1
63657.26         1
12635.11         1
10326.63         1
12899.07         1
1135.73          1
54.92            1
2363.44          1
13275.00         1
389.02           1
26549.00         1
280.13           1
272.31           1
229.85           1
273.69           1
53099.00         1
106198.50        1
57440.00         1
10029.37         1
30740.00    

'<usd_goal_real>'

5000.00      24173
10000.00     20786
1000.00      13029
3000.00      12699
2000.00      11915
15000.00     11374
20000.00     10121
2500.00       9849
500.00        8588
25000.00      8364
1500.00       8171
50000.00      7230
4000.00       6688
6000.00       5983
30000.00      5908
3500.00       5695
8000.00       5215
7000.00       4115
7500.00       4076
12000.00      4074
100000.00     3480
35000.00      2709
1200.00       2493
40000.00      2476
5500.00       2382
4500.00       2328
300.00        2210
600.00        2059
800.00        1919
6500.00       1877
             ...  
51918.94         1
1756.41          1
5568.56          1
2372.80          1
792.52           1
19990.77         1
22279.99         1
15000.15         1
3085.00          1
45211.36         1
30017.45         1
12684.31         1
10040.99         1
3170.58          1
330509.31        1
22835.57         1
3244.59          1
2770.31          1
19980.27         1
10327.08         1
33691.64         1
878.83      

In [85]:
# 欠損値を含むレコードを削除
print( '◆欠損値を含むレコード（削除前）' )
print( df_ks.isnull().sum() )
df_ks = df_ks.dropna()
print( '◆欠損値を含むレコード（削除後）' )
print( df_ks.isnull().sum() )

◆欠損値を含むレコード（削除前）
ID                     0
name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64
◆欠損値を含むレコード（削除後）
ID                  0
name                0
category            0
main_category       0
currency            0
deadline            0
goal                0
launched            0
pledged             0
state               0
backers             0
country             0
usd pledged         0
usd_pledged_real    0
usd_goal_real       0
dtype: int64


In [86]:
# 'state' の値を failed->False, successful ->True に置き換える
display( df_ks['state'].value_counts() )
df_ks.loc[ df_ks['state'] == 'failed', 'state'] = False
df_ks.loc[ df_ks['state'] == 'successful', 'state'] = True

# 'state' の値が False, True いずれでもない行は削除する
drop_index = df_ks.index[ ( df_ks['state'] != False ) & ( df_ks['state'] != True ) ]
df_ks = df_ks.drop( drop_index )
display( df_ks['state'].value_counts() )

failed        197611
successful    133851
canceled       38757
live            2798
suspended       1843
Name: state, dtype: int64

False    197611
True     133851
Name: state, dtype: int64

In [77]:
# backers と usd pledged を説明変数として、state を目的変数として学習
X = df_ks[['backers', 'usd pledged']].values
y = df_ks['state'].values
clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
clf.fit(X, y)

# 重み付けの出力
w0 = clf.intercept_[0]
w1 = clf.coef_[0, 0]
w2 = clf.coef_[0, 1]
print("w0 = {:.3f}, w1 = {:.3f}, w2 = {:.3f}".format(w0, w1, w2))

w0 = -26845.522, w1 = 339.349, w2 = 7.304


In [93]:
# 予測
y_est = clf.predict(X)

# 対数尤度を表示
print('対数尤度 = {:.3f}'.format(- log_loss(y, y_est)))

# 正答率を表示
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y, y_est)))

# Precision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_est)

# Precision, Recall, F1-scoreを表示
print('適合率（Precision） = {:.3f}%'.format(100 * precision[1]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[1]))
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[1]))

対数尤度 = -6.191
正答率 = 82.075%
適合率（Precision） = 81.643%
再現率（Recall） = 71.742%
F1値（F1-score） = 76.373%


In [94]:
# 予測値と正解のクロス集計
conf_mat = pd.DataFrame(confusion_matrix(y, y_est), index=['正解 = 0', '正解 = 1'], columns=['予測値 = 0', '予測値 = 1'])
conf_mat

,予測値 = 0,予測値 = 1
正解 = 0,176020,21591
正解 = 1,37824,96027
